# Lab 9: Audio Classification 

In [1]:
# imports
import numpy as np
from matplotlib import pyplot as plt
from scipy.io.wavfile import read, write
from scipy.signal import hann
import os

# soundplayer function
def sound( x, rate=44100, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))

In [2]:
# spectrograph function
dft_size_dflt = 2000
hop_size_dflt = 1000
def stft( input_sound, numb_bins=5000, dft_size=dft_size_dflt, hop_size=hop_size_dflt, window=0):
    if (numb_bins==0):
        numb_bins=dft_size//2+1
    if (window==0):
        window=hann(dft_size)
    if (input_sound.size%hop_size) != 0:
        input_sound = np.pad(input_sound, (0, hop_size-(input_sound.size%hop_size))) # pad input to hop size so last frame is full
    f = np.zeros(numb_bins) # initial matrix column
    current = 0
    while current <= (input_sound.size-dft_size):
        arg = input_sound[current:current+dft_size]
        dft = np.fft.rfft(window*arg, (numb_bins-1)*2)
        f = np.column_stack((f, dft))
        current += hop_size
    return f[:, 1:] # complex-valued spectrogram - frequencies(rows) x time(columns)

In [3]:
# inverse spectrograph function
def istft( stft_output, dft_size=dft_size_dflt, hop_size=hop_size_dflt):
    numb_bins = stft_output.shape[0] # number of frequency bins in spectrogram
    numb_dft = stft_output.shape[1] # number of dft slices in spectrogram
    f = np.zeros((numb_bins-1)*2 + (numb_dft-1)*hop_size) # output array
    current = 0
    while current < (numb_dft):
        arg = stft_output[:, current]
        idft = np.fft.irfft(arg)
        f[current*hop_size : current*hop_size + idft.size] += idft
        current+=1
    return f[:dft_size+(numb_dft-1)*hop_size] # real-valued array of audio samples in time-domain

## Part 1: Making a speech detector

In this section we will design a simple classifier that will let us know if its input is speech or non-speech. Download the data archive from: [ https://drive.google.com/file/d/1oAnvk-hzzgzZ4di4W0pKw6v3IWLm9u2X/view?usp=sharing ] In this part we will use the dataset in data/SpeechMusic. In it you will find two directories, speech/ and music/ containing data from each class.

Randomly select 50 soundfiles from each directory to use as training data, and use the remaining sounds as testing data. For all of the sounds we will compute a representation that makes the classification easier and we will use a simple Gaussian model to classify them. Do the following:

- Perform an STFT for each sound, take it’s magnitude and raise it to 0.3 to improve contrast
    - We will consider each spctral slice of that to be a data point
- Using the training data of each sound:
    - Calculate the mean column and the diagonal covariance of the columns
    - You will thus get two sets of Gaussian parameters that model each sound class
- For each testing data point:
    - Calculate the likelihood of each column based on the above models
	- To calculate the entire file likelihood add all the frame likelihoods
	- Assign each soundfile to the class that gets the highest likelihood

For extra credit implement the parameter estimation and model likelihood yourself. If you are too lazy for that you can instead use ```sklearn.mixture.GaussianMixture``` to learn a diagonal single-Gaussian model per class.

How do the results look like? If you rerun this with a different training/testing set, is there an appreciable difference? On average over multiple training/testing sets what accuracy do you get?

In [4]:
# learning about generators
#gen = os.walk('data/SpeechMusic/')
#print(gen)
#print(next(gen))
#print(next(gen))
#print(next(gen))

In [5]:
# load sounds
music_sr = np.empty(0)
music = []
for root, sub, files in os.walk('data/SpeechMusic/music/'):
    for f in files:
        sr, x = read(os.path.join(root, f))
        music_sr = np.append(music_sr, sr)
        music.append(x)
print(music_sr.shape, len(music))

speech_sr = np.empty(0)
speech = []
for root, sub, files in os.walk('data/SpeechMusic/speech/'):
    for f in files:
        sr, x = read(os.path.join(root, f))
        speech_sr = np.append(speech_sr, sr)
        speech.append(x)
print(speech_sr.shape, len(speech))

(60,) 60
(60,) 60


In [6]:
# divide up the sets into training/test data
split = 50
music_train = music[:split]
music_test = music[split:]
speech_train = speech[:split]
speech_test = speech[split:]

print(len(music_train), len(music_test))
print(len(speech_train), len(speech_test))

50 10
50 10


In [7]:
# check sample rates
#print(music_sr)
#print(speech_sr)
sr = 22050

# listen to the sounds
sound(music_train[0], sr)
sound(music_test[0], sr)
sound(speech_train[0], sr)
sound(speech_test[0], sr)

In [8]:
# stft
music_stft = []
for i in range(len(music)):
    f = stft(music[i])
    music_stft.append(f)

speech_stft = []
for i in range(len(speech)):
    f = stft(speech[i])
    speech_stft.append(f)
    
print(len(music_stft))
print(len(speech_stft))

print(music_stft[0].shape)
print(speech_stft[0].shape)

60
60
(5000, 330)
(5000, 330)


## Part 2: Making a music genre classifier

We will repeat the above, but this time we will perform music genre classification. To do so we will use a slightly more elaborate feature representation, and a stronger classification model. If you downloaded the data archive pointed to above, you will find a subset of the CTZAN dataset in the data/genre folder, this is a benchmark data set for music genre classification.

Just as before, you will find a set of directories with examples of each sound class that we want to recognize. For each class, split the soundfiles into a training set (50% of data) and testing set (remaining 50% of data).

For a representation we will use MFCC features. For extra credit, code these yourself otherwise you can use the implementation from the ```librosa``` library. Once all the files are transformed we will have a series of MFCC frames for each recording (as opposed to spectral frames as is in the case of the STFT). We will use these as the data to classify.

For each class learn a Gaussian model (with a diagonal covariance again). This will be the same process as above.
In order to evaluate how good this works we will use the following procedure. For each sound in the training data, get the likelihood of each MFCC frame based on the learned Gaussian models and sum these over the entire file just as we did before. Use the resulting values to get a classification result for each . Report how accurate your results are. Now report the accuracy using your testing data instead.

Now will use a better classifier to hopefully get better accuracy. We will use a Gaussian Mixture Model (```sklearn.mixture.GaussianMixture```). Just as before you should learn one such model for each class using the corresponding training data.

How many Gaussians do you need in your GMM to get the best results? Do the MFCC parameters make a difference? Play around with the numbers to get the best possible results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part 3: Make it better (extra credit, required for 4-hour registrants)

There is no shortage of techniques (and free code) to use for classification. Revisit the two problems above and use any other type of classifier you want (Neural Nets, Boosting, Decision Trees, whatever). Also feel free to use any feature you want. Can you improve on the results you got before? How much higher can you get your accuracy for either case?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()